In [19]:
import warnings

from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from bayes_opt import BayesianOptimization

from utils import preprocess_data

In [20]:
warnings.filterwarnings("ignore")

X_train, X_test, y_train, y_test, train_df, test_df = preprocess_data(standardise=True)

In [21]:
def optimize_knn(n_neighbors, leaf_size, p):
    model = KNeighborsClassifier(
        n_neighbors=int(n_neighbors),
        leaf_size=int(leaf_size),
        p=int(p)
    )
    return cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()


In [22]:
param_space = {
    'n_neighbors': (1, 30),
    'leaf_size': (20, 40),
    'p': (1, 2)  # Minkowski metric parameter (1 for Manhattan, 2 for Euclidean)
}


In [23]:
optimizer = BayesianOptimization(
    f=optimize_knn,
    pbounds=param_space,
    random_state=42
)

# Start the optimization process
optimizer.maximize(init_points=5, n_iter=55)


|   iter    |  target   | leaf_size | n_neig... |     p     |
-------------------------------------------------------------
| 1         | 0.7637    | 27.49     | 28.57     | 1.732     |
| 2         | 0.7901    | 31.97     | 5.525     | 1.156     |
| 3         | 0.7698    | 21.16     | 26.12     | 1.601     |
| 4         | 0.7475    | 34.16     | 1.597     | 1.97      |
| 5         | 0.7962    | 36.65     | 7.158     | 1.182     |
| 6         | 0.7901    | 32.0      | 5.591     | 1.182     |
| 7         | 0.7962    | 35.05     | 9.67      | 1.0       |
| 8         | 0.7983    | 39.27     | 10.27     | 1.0       |
| 9         | 0.7942    | 26.67     | 6.753     | 1.0       |
| 10        | 0.7983    | 37.79     | 15.72     | 2.0       |
| 11        | 0.7841    | 32.11     | 15.94     | 1.0       |
| 12        | 0.7739    | 40.0      | 20.5      | 1.0       |
| 13        | 0.784     | 22.25     | 3.0       | 1.0       |
| 14        | 0.7983    | 36.91     | 12.67     | 2.0       |
| 15    

In [24]:
best_params = optimizer.max['params']
best_model = KNeighborsClassifier(
    n_neighbors=int(best_params['n_neighbors']),
    leaf_size=int(best_params['leaf_size']),
    p=int(best_params['p'])
)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized KNN Accuracy: {accuracy}")


Optimized KNN Accuracy: 0.7967479674796748


In [25]:
model = KNeighborsClassifier()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized KNN Accuracy: {accuracy}")

Optimized KNN Accuracy: 0.7560975609756098
